In [ ]:
import csv
import subprocess
import shlex
import json

import numpy as np

from io import StringIO
from matplotlib import pyplot as plt
from scipy.stats import chisquare, kstest, norm, combine_pvalues
from IPython.display import display, Markdown

In [ ]:
target_directory = json.loads(subprocess.run("cargo metadata --format-version 1".split(), capture_output=True).stdout)["target_directory"]

# Normal Dispersal Kernel Test

In [ ]:
def test_normal_dispersal_kernel(
    algorithm, speciation=0.001, seeds=[42], sample=1.0, radius=0, sigma=100.0,
):
    total_pvalues_x = []
    total_pvalues_y = []
    total_location_deltas = []
    
    for seed in seeds:
        # Configure the simulation
        config = "".join("""
        (
            speciation: {speciation},
            seed: {seed},
            sample: {sample},

            algorithm: {algorithm}(),

            scenario: AlmostInfinite(
                radius: {radius},
                sigma: {sigma},
            ),

            reporters: [
                Plugin(
                    library: "{target_directory}/release/deps/libnecsim_plugins_statistics.so",
                    reporters: [
                        GlobalCoverage(output: "{output_file}"),
                    ],
                ),
            ],
        )
        """.format(
            target_directory=target_directory, output_file="/dev/fd/2",
            algorithm=algorithm, speciation=speciation, seed=seed, sample=sample,
            radius=radius, sigma=sigma,
        ).split()).replace(",)", ")").replace(",]", "]")

        # Run the simulation
        locations_io = StringIO(subprocess.run(shlex.split(
            "cargo run --release --features rustcoalescence-algorithms-monolithic,"
            + f"rustcoalescence-algorithms-independent --quiet -- simulate '{config}'"
        ), check=True, capture_output=True, text=True).stderr)

        # Read in the dispersal locations
        location_deltas = []
        with locations_io:
            reader = csv.reader(locations_io)
            next(reader)

            for row in reader:
                location_deltas.append((int(row[3]) - int(row[0]), int(row[4]) - int(row[1])))
        location_deltas = np.array(location_deltas)

        np.random.seed(seed & np.uint64(0xFFFFFFFF))
        samples = np.round(np.random.normal(loc=0.0, scale=sigma, size=(len(location_deltas), 2)))

        # Calculate the goodness of fit of the dispersal kernel
        gof_x = kstest(location_deltas[:,0], samples[:,0])
        gof_y = kstest(location_deltas[:,1], samples[:,1])
        
        total_pvalues_x.append(gof_x.pvalue)
        total_pvalues_y.append(gof_y.pvalue)
        total_location_deltas.append(location_deltas)
    
    gof_x_statistic, gof_x_pvalue = combine_pvalues(total_pvalues_x)
    gof_y_statistic, gof_y_pvalue = combine_pvalues(total_pvalues_y)
    
    if gof_x_pvalue <= 0.01 or gof_x_pvalue >= 0.99 or gof_y_pvalue <= 0.01 or gof_y_pvalue >= 0.99:
        display(Markdown(f"## <span style='color:purple'><u>{algorithm}</u></span>"))
    elif gof_x_pvalue <= 0.05 or gof_x_pvalue >= 0.95 or gof_y_pvalue <= 0.05 or gof_y_pvalue >= 0.95:
        display(Markdown(f"## <span style='color:red'><u>{algorithm}</u></span>"))
    elif gof_x_pvalue <= 0.1 or gof_x_pvalue >= 0.9 or gof_y_pvalue <= 0.1 or gof_y_pvalue >= 0.9:
        display(Markdown(f"## <span style='color:orange'>*{algorithm}*</span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{algorithm}</span>"))
        
    display(Markdown("#### Fisher’s combined KS test:"))
    display(Markdown(f"* x-axis p-value: {gof_x_pvalue}\n* x-axis statistic: {gof_x_statistic}"))
    display(Markdown(f"* y-axis p-value: {gof_y_pvalue}\n* y-axis statistic: {gof_y_statistic}"))
    
    display(Markdown("#### Dispersal Histograms:"))
    
    location_deltas = np.concatenate(total_location_deltas)
    
    sigma_3 = int(3*sigma)
    step = int(max(sigma_3*2 / 100, 1))
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
    
    ax1.set_title("Per-generation relative dispersal")
    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    hist1 = ax1.hist2d(location_deltas[:,0], location_deltas[:,1], bins=(range(-sigma_3, sigma_3+1, step), range(-sigma_3, sigma_3+1, step)), density=True)
    fig.colorbar(hist1[3], ax=ax1)
    
    x = np.linspace(-sigma_3, sigma_3, 2*sigma_3 + 1)
    p = norm.pdf(x, loc=0.0, scale=sigma)
    
    ax2.set_title("Dispersal histogram x")
    ax2.set_xlabel("x")
    ax2.set_ylabel("pmf")
    ax2.plot(x, p, color='red', linewidth=2, alpha=0.5, label=f"expected: mu=0.0 sigma={sigma}")
    loc, scale = norm.fit(location_deltas[:,0])
    ax2.hist(location_deltas[:,0], bins=range(-sigma_3, sigma_3+1), density=True, label=f"observed: mu≈{round(loc, 3)} sigma≈{round(scale, 3)}")
    ax2.legend()
    ax2.legend(loc='lower center')
    
    ax3.set_title("Dispersal histogram y")
    ax3.set_xlabel("y")
    ax3.set_ylabel("pmf")
    ax3.plot(x, p, color='red', linewidth=2, alpha=0.5, label=f"expected: mu=0.0 sigma={sigma}")
    loc, scale = norm.fit(location_deltas[:,1])
    ax3.hist(location_deltas[:,1], bins=range(-sigma_3, sigma_3+1), density=True, label=f"observed: mu≈{round(loc, 3)} sigma≈{round(scale, 3)}")
    ax3.legend(loc='lower center')
    
    plt.show()
    
    display(Markdown(f"#### Parameters:\n* seeds: {seeds}"))
    
    display(Markdown(f"#### Configuration:\n```rust\n{config}\n```"))

In [ ]:
for algorithm in ["Classical", "Gillespie", "SkippingGillespie", "Independent"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=10)
    
    test_normal_dispersal_kernel(
        algorithm, speciation=0.001, seeds=seeds, sample=1.0, radius=25, sigma=100.0
    )